# Step 1 - call a model (make sure to set your open ai key)

In [ ]:
import duckdb
import openai

DEFAULT_MODEL = "gpt-4o-2024-08-06"  # "gpt-4-1106-preview"
GPT_MINI = "gpt-4o-mini"
prompt = "help the user out"

question = "what is the best thing about the dutch"
messages = [
     {"role": "system", "content": prompt},
     {"role": "user", "content": question}
]

response = openai.chat.completions.create(
        model=GPT_MINI,
        messages=messages,
        temperature=0,
        response_format=None, #can be json like,
        stream=None
    )
    
response.choices[0].message.content

# Step 2 - call a function

In [ ]:
"""
there are probably lots of ways to do this - i went first principles to show the idea 
and assume (brittle) a doc string so we dont need to much code for handling types etc
"""

def describe_function(fn):
    """this is a simplistic thing that generates openai schema for functions from doc strings
       this is not what i do but its a way to do it fast and since what happens
    """
    import docstring_parser
    a=docstring_parser.parse(fn.__doc__)

    return {
        "name": fn.__name__, 
        "description": a.description,
        "parameters": {
            "type": "object",
            "properties": { p.arg_name :{"type":p.type_name, "description": p.description}  for p in a.params }
        }
    }



In [ ]:
question = "can you call my function please and pass hi as the param"
messages = [
     {"role": "system", "content": prompt},
     {"role": "user", "content": question}
]

def my_function(param: str):
    """
    this function spits out a param
    
    Args:
        param(string): the param 
    """
    
    return f"You said {param} and its nice to meet you"

"""
if you were super strict with doc strings this would work
""" 
functions = [describe_function(my_function)]

response = openai.chat.completions.create(
        model=GPT_MINI,
        functions=functions,
        messages=messages,
        temperature=0,
        response_format=None, #can be json like,
        stream=None
    )

fcall = response.choices[0].message.function_call
fcall

In [ ]:
import json
def invoker(call):
    """you can call using the openapi object
       parse the json arguments with your checks etc
       handle exceptions and all that jazz
       generally try to return a json response
    """
    return {
        "status": "you have called the function",
      f"response from function {fcall.name}" : eval(call.name)(**json.loads(call.arguments)) 
    } 

In [ ]:
invoker(fcall)

In [ ]:
# 3 understand the states in the loop and look for stoping conditions

question = "what was the result of calling the function with parameter Ireland"
messages = [
     {"role": "system", "content": "Use any functions you have to answer the users question"},
     {"role": "user", "content": question}
]


for i in range(5):
    """fixed function list above, i usually dynmically add functions"""
    response = openai.chat.completions.create(
        model=GPT_MINI,
        functions=functions,
        messages=messages,
        temperature=0,
        response_format=None, #can be json like,
    )
    fcall = response.choices[0].message.function_call
    if fcall:
        """note the role is a funciton now"""
        messages.append({
            'role': 'function',
            'name': fcall.name,
            'content': json.dumps(invoker(fcall), default=str)
        })
        """messages are now updated """
    else:
        pass
    if response.choices[0].finish_reason == 'stop':
        break

response.choices[0].message.content

In [ ]:
from pathlib import Path

# uri = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet'
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet ~/Downloads
file = f"{Path.home()}/Downloads/yellow_tripdata_2024-01.parquet"


def get_query(question, file, extra_hints: str= None):
    """"""
    
    """probe for schema and enums etc..."""
    data = duckdb.execute(f" select * from '{file}' limit 1")
    columns = dict(data.df().dtypes)
    
    prompt= f"""
    Please provide an sql query using Duck DB dialect to answer the users question for a table called TABLE The schema for the data are
    ```json
    {columns}
    ```
    
    You can response using this model in Json
    
    class MyModel(BaseModel):
        sql: str
        comment: str
        
    ### hints
    ```
    {extra_hints}
    ```
    """
    
    messages = [
         {"role": "system", "content": prompt},
         {"role": "user", "content": question}
    ]

    response = openai.chat.completions.create(
            model=GPT_MINI,
            messages=messages,
            temperature=0,
            response_format=  {"type": "json_object"}
        )

    data =  json.loads(response.choices[0].message.content)
    data['sql'] = data['sql'].replace('TABLE', f"'{file}'")
    print(data['sql'])
    return data

query = get_query('what were the total surcharges', file)

duckdb.execute(query['sql']).df()

In [ ]:
query = get_query('please provide 10 sample rows', file)
duckdb.execute(query['sql']).df()

In [ ]:
#breaks down
query = get_query('what day had the largest congestion surcharge', file)
duckdb.execute(query['sql']).df()

In [ ]:
hints = """## Working with duckdb dialect

 ### 1 some example *list* column type functions
    - Un-nesting list types: You can use the UNEST function as per the following example taking care of column names and alias if you want to perform aggregations on list types.

```sql
    -- use CTEs with group by to avoid aliasing confusion - this example unnests a list
    WITH unnested_sku AS (
            SELECT   UNNEST(skus) AS individual_sku  FROM  TABLE
        )
        SELECT    individual_sku,  COUNT(*) AS frequency  FROM  unnested_sku
        GROUP BY   individual_sku
        ORDER BY frequency
```

### 2 Here are some example *date functions* should you need them
- truncate to precision using `date_trunc(part,date_column)` for example to get the date only  `date_trunc('day', date_column)`
- we can get date differences and sample parts with - `date_diff(part, startdate, enddate)`
- when asked date based questions be liberal with the dates e.g. treat "this week" as the last 7 days or this month as the last 4 weeks
- do not use `date_sub` in duckdb queries - see example below for adding negative offsets
- casting dates e.g. DATE '1992-03-22' (but you can assume dates are already in the correct type and not strings by default)
- do not use the function `NOW()` use `current_date()` instead for the current date time

#### Some other examples

**Add or Subtract the interval to the date using `date_add(date, interval)`. Note we add negative numbers for subtraction**
- For example,	```date_add(DATE '1992-09-15', INTERVAL 2 MONTH)``` results in `1992-11-15`
- Subtraction uses ` - INTERVAL` e.g.  	```date_add(DATE '1992-09-15', - INTERVAL 3 MONTH``` results in `1992-06-15`

**Getting the number of partition boundaries between the dates**
-	`date_diff('month', DATE '1992-09-15', DATE '1992-11-14')`

**Get the subfield (equivalent to extract) of the date using `date_part(part,date)`**
- 	`date_part('year', DATE '1992-09-20')` results in `1992`"""

query = get_query('what day had the largest congestion surcharge', file, extra_hints=hints)
duckdb.execute(query['sql']).df()

In [ ]:
query = get_query('what are all the payment types', file, extra_hints=hints)
duckdb.execute(query['sql']).df()

In [ ]:
query = get_query('what was the average distance travelled', file, extra_hints=hints)
duckdb.execute(query['sql']).df()